In [91]:
import pandas as pd
import numpy as np
from datetime import datetime
danver_path = "./danvercrime.csv"
df = pd.read_csv(danver_path)[:10]

In [92]:
def time_cov(raw_in):
    if type(raw_in)==float:
        return raw_in,raw_in,raw_in,raw_in,raw_in,raw_in
    datetime_raw = datetime.strptime(raw_in, '%m/%d/%Y %I:%M:%S %p')
    date = int(datetime_raw.strftime("%Y%m%d"))
    hour = datetime_raw.hour
    minute = datetime_raw.minute
    second = datetime_raw.second
    dow = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"][datetime_raw.weekday()]
    return date,hour,minute,second,dow,datetime_raw


In [111]:
fact_df = pd.DataFrame(columns=['location_Key','crime_key', 'date_key','event_holiday_key', 'is_traffic','is_fatal','is_nighttime'])
location_df = pd.DataFrame(columns=['location_Key','city', 'neighborhood','longitude', 'latitude'])
'''crime_df = pd.DataFrame(columns=['crime_key','crime_category_name', 'crime_type_name',
                                 'start_year', 'start_month',"start_day",'start_hour', 'start_minute',"start_second",
                                 'end_year', 'end_month',"end_day",'end_hour', 'end_minute',"end_second",
                                 'report_year', 'report_month',"report_day",'report_hour', 'report_minute',"report_second"])'''
crime_df= pd.DataFrame(columns=['crime_key','crime_category_name', 'crime_type_name',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second"])
date_df = pd.DataFrame(columns=['date_key','day_of_week','year','month','day'])
holiday_event_df = pd.DataFrame(columns=['event_holiday_key','event_holiday_name','event_holiday_start_time','event_holiday_end_time','is_statutory_holiday','is_weekend'])


In [ ]:
# TODO
statutory_holiday_dict = [
    {"name":"New Year's Day",
     "date": 101,
     "fixed_date":True,
     "applied_year":None},
    {"name":"Thanksgiving",
     "date": 1128,
     "apply_in_every_year":False,
     "year":2019}
]
event_dict={}

In [114]:
for index, row in df.iterrows():
    location_id = index + 1000000
    city = "Danver"
    neighborhood = row["NEIGHBORHOOD_ID"]
    lat = row["GEO_LAT"]
    lon = row["GEO_LON"]
    location_df.loc[index] = [location_id,city,neighborhood,lon,lat]
    
    
    crime_id = index + 2000000
    crime_category_name = row["OFFENSE_CATEGORY_ID"]
    crime_type_name = row["OFFENSE_TYPE_ID"]
    s_d,s_h,s_m,s_s,s_dow,s_dt = time_cov(row["FIRST_OCCURRENCE_DATE"])
    e_d,e_h,e_m,e_s,e_dow,e_dt = time_cov(row["LAST_OCCURRENCE_DATE"])
    r_d,r_h,r_m,r_s,r_dow,r_dt = time_cov(row["REPORTED_DATE"])
    crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,s_h,s_m,s_s,e_h,e_m,e_s,r_h,r_m,r_s]
    
    
    # TODO Full Date dimension
    date_id = index + 3000000
    date_df.loc[index] = [date_id,s_dow,s_dt.year,s_dt.month,s_dt.day]
    
    
    #TODO
    event_holiday_id = np.nan
    holiday=[]
    
    is_night = 1
    if 7 <= s_m <= 18:
        is_night = 0
    
    #TODO
    is_fatal = 0

    
    fact_df.loc[index] = [date_id,location_id,crime_id,event_holiday_id,row["IS_TRAFFIC"],is_fatal,is_night]

In [116]:
df.head()

,INCIDENT_ID,OFFENSE_ID,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC
0,2016376978,2016376978521300,5213,0,weapon-unlawful-discharge-of,all-other-crimes,6/15/2016 11:31:00 PM,NaN,6/15/2016 11:31:00 PM,NaN,3193983.0,1707251.0,-104.809881,39.773188,5,521,montbello,1,0
1,20186000994,20186000994239900,2399,0,theft-other,larceny,10/11/2017 12:30:00 PM,10/11/2017 4:55:00 PM,1/29/2018 5:53:00 PM,NaN,3201943.0,1711852.0,-104.781434,39.785649,5,522,gateway-green-valley-ranch,1,0
2,20166003953,20166003953230500,2305,0,theft-items-from-vehicle,theft-from-motor-vehicle,3/4/2016 8:00:00 PM,4/25/2016 8:00:00 AM,4/26/2016 9:02:00 PM,2932 S JOSEPHINE ST,3152762.0,1667011.0,-104.957381,39.663490,3,314,wellshire,1,0
3,201872333,201872333239900,2399,0,theft-other,larceny,1/30/2018 7:20:00 PM,NaN,1/30/2018 10:29:00 PM,705 S COLORADO BLVD,3157162.0,1681320.0,-104.941440,39.702698,3,312,belcaro,1,0
4,2017411405,2017411405230300,2303,0,theft-shoplift,larceny,6/22/2017 8:53:00 PM,NaN,6/23/2017 4:09:00 PM,2810 E 1ST AVE,3153211.0,1686545.0,-104.955370,39.717107,3,311,cherry-creek,1,0


In [109]:
crime_df.head()

,crime_key,crime_category_name,crime_type_name,report_hour,report_minute,report_second,start_hour,start_minute,start_second,end_hour,end_minute,end_second
0,2000000,all-other-crimes,weapon-unlawful-discharge-of,23,31,0,23,31,0,NaN,NaN,NaN
1,2000001,larceny,theft-other,17,53,0,12,30,0,16,55,0
2,2000002,theft-from-motor-vehicle,theft-items-from-vehicle,21,2,0,20,0,0,8,0,0
3,2000003,larceny,theft-other,22,29,0,19,20,0,NaN,NaN,NaN
4,2000004,larceny,theft-shoplift,16,9,0,20,53,0,NaN,NaN,NaN
